In [8]:
#in this file, I will create and save a variety of new dataframes with new features
#these features are engineered features, designed in the hope of improving model performance 
#at this stage, I am still only looking at the first timeslice of the data. This file will be within that context. 
#The output dataframes will be the first (full, including where there is no ice) time slice, plus the extra features. 

import pandas as pd 

df = pd.read_csv('../data/AIS-86Year/vars_all_v2/vars-2015.txt', delimiter = '\t',
                      names=["x-axis", "y-axis", "ice_thickness", "ice_velocity", "ice_mask", 
                             "precipitation", "air_temp", "ocean_temp"])

def scale (df):
    """uses the formula scaled val = (val-column minimum)/(column maximum - column minimum)"""
    scaled_df = df.copy()
    # for column in df.columns[:-1]  -> use this line instead for not having a scaled target 
    for column in scaled_df.columns:
        min_value = scaled_df[column].min()
        max_value = scaled_df[column].max()
        scaled_df[column] = (scaled_df[column] - min_value) / (max_value - min_value)
            
    return scaled_df

df = scale(df)
df.head()

,x-axis,y-axis,ice_thickness,ice_velocity,ice_mask,precipitation,air_temp,ocean_temp
0,0.00,1.0,0.0,NaN,1.0,0.320249,0.958077,1.0
1,0.02,1.0,0.0,NaN,1.0,0.305467,0.730157,1.0
2,0.04,1.0,0.0,NaN,1.0,0.304381,0.724929,1.0
3,0.06,1.0,0.0,NaN,1.0,0.298838,0.718765,1.0
4,0.08,1.0,0.0,NaN,1.0,0.290624,0.712535,1.0


In [ ]:
#creating a feature to represent the distance from pole of each point 



In [ ]:
#this feature represents the difference between ocean and air temperature 
#i thought it could aid in representing the interaction between atmposphere and sea, which is an important 
#factor in the behaviour of the ice sheet 